<a href="https://colab.research.google.com/github/davidlealo/TalkTrack/blob/main/analisis_seminario_colunga14_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transcribir texto

In [1]:
# Instalar whisper y otros paquetes necesarios
!pip install -q whisper openai ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [ ]:
from google.colab import files

# Subir un archivo de audio desde tu computadora
uploaded = files.upload()

# Extraer el nombre del archivo subido
audio_file = list(uploaded.keys())[0]
print(f"Archivo de audio subido: {audio_file}")


In [ ]:
import whisper

# Cargar el modelo de Whisper (puedes usar 'base', 'small', 'medium', 'large' según tu necesidad)
model = whisper.load_model("base")

# Transcribir el audio
result = model.transcribe(audio_file)

# Imprimir la transcripción
transcription = result["text"]
print("Transcripción:\n", transcription)


In [ ]:
# Guardar la transcripción en un archivo .txt
output_file = "transcription.txt"
with open(output_file, "w") as f:
    f.write(transcription)

print(f"Transcripción guardada en: {output_file}")


## Análisis de discurso

In [ ]:
# Instalar las bibliotecas necesarias
!pip install anthropic


In [ ]:
# Importar las bibliotecas necesarias
import os
from anthropic import Client
from IPython.display import display, HTML
from google.colab import output

# Interfaz para ingresar la API key
def request_anthropic_api_key():
    input_html = """
    <div style="font-family: Arial; margin: 10px; padding: 10px; border: 1px solid #ddd; border-radius: 5px; width: 50%;">
        <h3 style="color: #007BFF;">Proveer API Key</h3>
        <p>Por favor, ingrese su API key de Anthropics en el siguiente campo:</p>
        <input type="password" id="anthropicApiKeyInput" placeholder="Ingrese su API key aquí"
               style="width: 100%; padding: 8px; margin-bottom: 10px; border: 1px solid #ddd; border-radius: 3px;">
        <button onclick="storeApiKey()"
                style="padding: 10px 15px; background-color: #007BFF; color: white; border: none; border-radius: 3px; cursor: pointer;">
            Guardar API Key
        </button>
    </div>
    <script>
        function storeApiKey() {
            const apiKey = document.getElementById('anthropicApiKeyInput').value;
            if (apiKey) {
                google.colab.kernel.invokeFunction('notebook.store_anthropic_api_key', [apiKey], {});
                alert('API key almacenada correctamente.');
            } else {
                alert('Por favor, ingrese una API key válida.');
            }
        }
    </script>
    """
    display(HTML(input_html))

# Callback para almacenar la API key en la variable de entorno
def store_anthropic_api_key(api_key):
    os.environ["ANTHROPIC_API_KEY"] = api_key
    print("API key almacenada correctamente en la variable de entorno.")

# Registrar la función en Google Colab
output.register_callback('notebook.store_anthropic_api_key', store_anthropic_api_key)

# Solicitar la API key al usuario
request_anthropic_api_key()

# Crear un cliente para la API
client = Client(api_key=os.environ["ANTHROPIC_API_KEY"])

def analyze_discourse(text):
    """
    Realiza un análisis de discurso usando la API de Anthropic.
    """
    # Definir un prompt ajustado al contexto del análisis
    prompt = f"""
    You are tasked with analyzing the discourse of an audio transcript from a seminar on innovating and transforming the classroom. Your goal is to provide insights into the content, structure, and key themes of the seminar.

    Here is the transcript of the audio:

    <transcript>
    {text}
    </transcript>

    Analyze the discourse of this seminar, focusing on the following areas:

    1. Main themes and topics discussed
    2. Innovative ideas or strategies presented
    3. Structure and flow of the seminar
    4. Speaker's communication style and effectiveness
    5. Audience engagement (if applicable)
    6. Use of examples or case studies
    7. Potential impact on classroom transformation

    Provide your analysis in a structured format with the following sections:

    <analysis>
    <themes>
    Discuss the main themes and topics of the seminar
    </themes>

    <innovation>
    Highlight the innovative ideas or strategies presented for classroom transformation
    </innovation>

    <structure>
    Analyze the structure and flow of the seminar
    </structure>

    <communication>
    Evaluate the speaker's communication style and effectiveness
    </communication>

    <engagement>
    Discuss any evidence of audience engagement or interaction
    </engagement>

    <examples>
    Highlight any significant examples or case studies used
    </examples>

    <impact>
    Assess the potential impact of the ideas presented on classroom transformation
    </impact>
    </analysis>

    In each section, use specific evidence from the transcript to support your analysis. Quote relevant parts of the transcript when appropriate, using quotation marks.

    After completing the structured analysis, provide a concise summary of your findings in a separate section:

    <summary>
    Summarize the key points of your analysis and the overall effectiveness of the seminar in addressing innovation and transformation in the classroom
    </summary>

    Remember to base your analysis solely on the content provided in the transcript. If certain aspects are not addressed in the transcript, you may note their absence in your analysis.
    """

    # Llamada a la API
    response = client.completion(
        prompt=prompt,
        model="claude-3-5-sonnet-20241022",
        max_tokens_to_sample=1000,
        stop_sequences=["\n\n"]
    )
    return response["completion"]

# Cargar el texto de la transcripción
with open("transcription.txt", "r") as f:
    transcription_text = f.read()

# Realizar el análisis
analysis = analyze_discourse(transcription_text)

# Mostrar el análisis
print("Análisis de Discurso:\n", analysis)

# Guardar el análisis en un archivo
with open("discourse_analysis.txt", "w") as f:
    f.write(analysis)

print("El análisis ha sido guardado en 'discourse_analysis.txt'.")


### Nubes de palabras

In [ ]:
!pip install wordcloud matplotlib


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def generate_wordcloud(text):
    wordcloud = WordCloud(width=800, height=400, background_color="white").generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title("Nube de Palabras del Discurso", fontsize=16)
    plt.show()

# Generar la nube de palabras a partir del texto de la transcripción
generate_wordcloud(transcription_text)


### Análisis de Sentimientos

In [ ]:
!pip install transformers


In [ ]:
from transformers import pipeline

# Crear pipeline de análisis de sentimientos
sentiment_analyzer = pipeline("sentiment-analysis")

# Dividir el texto en párrafos y analizar cada uno
sentiments = sentiment_analyzer(transcription_text.split("\n"))

# Mostrar los resultados
for idx, sentiment in enumerate(sentiments):
    print(f"Párrafo {idx + 1}: {sentiment}")


### Gráficos para Análisis de Temas

In [ ]:
!pip install gensim pyLDAvis


In [ ]:
from gensim import corpora
from gensim.models.ldamodel import LdaModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# Preprocesar el texto
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

nltk.download("punkt")
nltk.download("stopwords")

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words("english"))
    return [word for word in tokens if word.isalnum() and word not in stop_words]

# Tokenizar la transcripción
tokens = preprocess_text(transcription_text)

# Crear un diccionario y un corpus
dictionary = corpora.Dictionary([tokens])
corpus = [dictionary.doc2bow(tokens)]

# Crear modelo LDA
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=10)

# Visualizar temas
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_vis)


### Visualización del Impacto

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

# Contar las palabras más frecuentes
word_counts = Counter(tokens)
most_common_words = word_counts.most_common(10)

# Gráfico de barras
words, counts = zip(*most_common_words)
plt.bar(words, counts)
plt.title("Palabras más frecuentes")
plt.xticks(rotation=45)
plt.show()


### Resumen Visual

Diagrama de Sankey

Un diagrama de Sankey es excelente para mostrar relaciones entre temas principales y secundarios o el flujo lógico de una narrativa.

In [ ]:
!pip install plotly


In [ ]:
import plotly.graph_objects as go

def create_sankey_chart(data):
    """
    Crea un diagrama de Sankey para representar visualmente el resumen del discurso.
    """
    # Definir los nodos (temas principales y secundarios)
    nodes = list(set([item for sublist in data for item in sublist]))
    node_indices = {node: i for i, node in enumerate(nodes)}

    # Definir las conexiones entre los nodos
    sources = [node_indices[source] for source, target, value in data]
    targets = [node_indices[target] for source, target, value in data]
    values = [value for source, target, value in data]

    # Crear el diagrama de Sankey
    fig = go.Figure(go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=nodes
        ),
        link=dict(
            source=sources,
            target=targets,
            value=values
        )
    ))

    fig.update_layout(title_text="Resumen Visual del Discurso", font_size=10)
    fig.show()

# Datos de ejemplo (modifica según el análisis del discurso)
sankey_data = [
    ("Innovación", "Uso de tecnología", 5),
    ("Innovación", "Ejemplos prácticos", 3),
    ("Uso de tecnología", "Herramientas digitales", 2),
    ("Uso de tecnología", "Aplicaciones móviles", 3),
    ("Ejemplos prácticos", "Casos reales", 4),
    ("Casos reales", "Impacto en el aula", 6),
]

# Crear el diagrama
create_sankey_chart(sankey_data)


Mapa Mental

Si prefieres un mapa mental, podemos usar networkx y matplotlib para generar un gráfico de nodos.

In [ ]:
!pip install networkx matplotlib


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def create_mind_map(data):
    """
    Crea un mapa mental para representar visualmente el discurso.
    """
    G = nx.DiGraph()

    # Agregar nodos y relaciones
    for source, target, _ in data:
        G.add_edge(source, target)

    # Crear el diseño del gráfico
    pos = nx.spring_layout(G, k=0.5, iterations=50)
    plt.figure(figsize=(12, 8))
    nx.draw_networkx(
        G, pos, with_labels=True, node_color="lightblue", edge_color="gray",
        font_size=10, font_weight="bold", node_size=3000
    )
    plt.title("Mapa Mental del Discurso", fontsize=16)
    plt.axis("off")
    plt.show()

# Usar los mismos datos de ejemplo
create_mind_map(sankey_data)


### Grafo con networkx y matplotlib


In [ ]:
!pip install networkx matplotlib

import networkx as nx
import matplotlib.pyplot as plt

def create_graph(data):
    """
    Crea un grafo simple para representar las conexiones entre temas.
    """
    G = nx.Graph()

    # Agregar nodos y relaciones desde los datos
    for source, target, weight in data:
        G.add_edge(source, target, weight=weight)

    # Crear un diseño para el grafo
    pos = nx.spring_layout(G, k=0.5)

    # Dibujar el grafo
    plt.figure(figsize=(12, 8))
    nx.draw(
        G, pos, with_labels=True, node_color="skyblue", edge_color="gray",
        font_size=10, font_weight="bold", node_size=3000
    )

    # Agregar etiquetas para los pesos de las conexiones
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=9)

    plt.title("Grafo de Temas del Discurso", fontsize=16)
    plt.axis("off")
    plt.show()

# Datos de ejemplo (tema principal, tema relacionado, peso de conexión)
graph_data = [
    ("Innovación", "Uso de tecnología", 5),
    ("Innovación", "Ejemplos prácticos", 3),
    ("Uso de tecnología", "Herramientas digitales", 4),
    ("Ejemplos prácticos", "Casos reales", 2),
    ("Casos reales", "Impacto en el aula", 6)
]

# Crear el grafo
create_graph(graph_data)


### Grafo Interactivo con pyvis


In [ ]:
!pip install pyvis


In [ ]:
from pyvis.network import Network

def create_interactive_graph(data):
    """
    Crea un grafo interactivo usando pyvis.
    """
    net = Network(notebook=True, height="750px", width="100%", bgcolor="#ffffff", font_color="black")

    # Agregar nodos y relaciones desde los datos
    for source, target, weight in data:
        net.add_node(source, title=source)
        net.add_node(target, title=target)
        net.add_edge(source, target, value=weight)

    # Configurar opciones del grafo
    net.set_options('''
    var options = {
      "nodes": {
        "color": {
          "border": "rgba(0,0,0,1)",
          "background": "rgba(135,206,250,1)"
        }
      },
      "edges": {
        "color": {
          "inherit": true
        },
        "smooth": {
          "type": "dynamic"
        }
      }
    }
    ''')

    # Mostrar el grafo
    net.show("graph.html")

# Crear el grafo interactivo
create_interactive_graph(graph_data)


## Datos Dinámicos:

Usa un modelo LLM para extraer automáticamente temas principales, relaciones, y pesos del discurso

In [ ]:
from anthropic import Client
import os

# Cliente de Anthropic
client = Client(api_key=os.environ["ANTHROPIC_API_KEY"])

def extract_topics_and_relations(text):
    """
    Extrae temas principales, relaciones y pesos del discurso usando un LLM.
    """
    prompt = f"""
    You are an expert in discourse analysis. Based on the following transcript, extract the main topics, their subtopics,
    and the relationships between them. Include a relevance score (1-10) for each relationship.

    Transcript:
    {text}

    Provide the output in the following format:
    [
        ("Main Topic", "Related Topic", Relevance Score),
        ...
    ]
    """
    response = client.completion(
        prompt=prompt,
        model="claude-3-5-sonnet-20241022",
        max_tokens_to_sample=1000,
        stop_sequences=["\n\n"]
    )
    return eval(response["completion"])

# Usar la función en la transcripción
with open("transcription.txt", "r") as f:
    transcription_text = f.read()

relations_data = extract_topics_and_relations(transcription_text)
print("Relaciones extraídas:\n", relations_data)


## Crear un Grafo Dinámico con los Datos Extraídos
Ahora que tenemos los datos generados, podemos usarlos para construir un grafo dinámico.

Grafo Dinámico con pyvis:

In [ ]:
from pyvis.network import Network

def create_dynamic_graph(data):
    """
    Crea un grafo interactivo usando datos generados por un modelo LLM.
    """
    net = Network(notebook=True, height="750px", width="100%", bgcolor="#ffffff", font_color="black")

    # Agregar nodos y relaciones
    for source, target, weight in data:
        net.add_node(source, title=source)
        net.add_node(target, title=target)
        net.add_edge(source, target, value=weight)

    # Configurar opciones del grafo
    net.set_options('''
    var options = {
      "nodes": {
        "color": {
          "border": "rgba(0,0,0,1)",
          "background": "rgba(135,206,250,1)"
        }
      },
      "edges": {
        "color": {
          "inherit": true
        },
        "smooth": {
          "type": "dynamic"
        }
      }
    }
    ''')

    # Mostrar el grafo
    net.show("dynamic_graph.html")

# Crear el grafo dinámico con los datos extraídos
create_dynamic_graph(relations_data)


### Nube de Palabras: Resalta Términos Frecuentes
Crea una nube de palabras para identificar rápidamente las palabras clave más utilizadas en el discurso.



In [ ]:
!pip install wordcloud matplotlib

from wordcloud import WordCloud
import matplotlib.pyplot as plt

def generate_wordcloud_from_topics(data):
    """
    Genera una nube de palabras basada en los temas y sus pesos.
    """
    # Crear un diccionario con temas y pesos
    topic_weights = {source: weight for source, _, weight in data}
    topic_weights.update({target: weight for _, target, weight in data})

    # Generar la nube de palabras
    wordcloud = WordCloud(width=800, height=400, background_color="white").generate_from_frequencies(topic_weights)

    # Mostrar la nube de palabras
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title("Nube de Palabras Basada en Temas", fontsize=16)
    plt.show()

# Generar la nube de palabras
generate_wordcloud_from_topics(relations_data)


## Mapa de Calor: Relación Entre Temas
Un mapa de calor (heatmap) muestra cómo de fuertes son las relaciones entre temas principales y secundarios.

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

def create_heatmap(data):
    """
    Genera un mapa de calor para visualizar las relaciones entre temas.
    """
    # Crear un DataFrame a partir de los datos
    df = pd.DataFrame(data, columns=["Tema Principal", "Tema Relacionado", "Peso"])

    # Pivotar la tabla para crear la matriz
    matrix = df.pivot("Tema Principal", "Tema Relacionado", "Peso")

    # Crear el mapa de calor
    plt.figure(figsize=(10, 8))
    sns.heatmap(matrix, annot=True, cmap="Blues", fmt=".1f")
    plt.title("Mapa de Calor de Relaciones entre Temas", fontsize=16)
    plt.show()

# Generar el mapa de calor
create_heatmap(relations_data)


## Análisis Temporal: Evolución de Temas
Si tu discurso está dividido en segmentos temporales (e.g., minutos o secciones), puedes analizar cómo evolucionan los temas a lo largo del tiempo.



In [ ]:
import matplotlib.pyplot as plt

def plot_topic_evolution(data, timestamps):
    """
    Visualiza la evolución de los temas principales en el tiempo.
    """
    # Crear un diccionario para contar la aparición de temas por timestamp
    topic_evolution = {topic: [0] * len(timestamps) for topic, _, _ in data}

    # Llenar el diccionario con datos
    for i, (topic, _, weight) in enumerate(data):
        topic_evolution[topic][i % len(timestamps)] += weight

    # Graficar la evolución
    plt.figure(figsize=(12, 6))
    for topic, values in topic_evolution.items():
        plt.plot(timestamps, values, label=topic)

    plt.title("Evolución de Temas en el Tiempo", fontsize=16)
    plt.xlabel("Tiempo")
    plt.ylabel("Relevancia")
    plt.legend()
    plt.show()

# Ejemplo de timestamps (ajusta según tu análisis)
timestamps = ["Inicio", "Mitad", "Final"]
plot_topic_evolution(relations_data, timestamps)


## Árbol Jerárquico: Relación Principal-Secundaria
Visualiza los temas en forma de árbol jerárquico, con los temas principales como nodos padres y los secundarios como hijos

In [ ]:
!pip install plotly

import plotly.graph_objects as go

def create_tree_map(data):
    """
    Genera un árbol jerárquico para mostrar relaciones entre temas.
    """
    labels = []
    parents = []

    for source, target, weight in data:
        labels.append(target)
        parents.append(source)

    fig = go.Figure(go.Treemap(
        labels=labels + ["Total"],
        parents=parents + [""],
        values=[weight for _, _, weight in data] + [sum(weight for _, _, weight in data)]
    ))

    fig.update_layout(title="Árbol Jerárquico de Temas")
    fig.show()

# Crear el árbol jerárquico
create_tree_map(relations_data)


## Predicciones y Análisis Contextual:
Usa un modelo de lenguaje para:

Generar predicciones sobre el impacto de los temas en el contexto del discurso (e.g., "¿Cómo impactará X en la educación?").

Identificar conexiones ocultas entre temas que no son evidentes en el análisis inicial.

In [ ]:
def generate_additional_insights(text):
    prompt = f"""
    Based on the following topics and relationships, provide additional insights or predictions:
    {relations_data}

    Insights should include:
    1. Hidden connections between topics.
    2. Future trends related to these topics.
    3. Recommendations for further exploration.

    Transcript:
    {text}
    """
    response = client.completion(
        prompt=prompt,
        model="claude-3-5-sonnet-20241022",
        max_tokens_to_sample=1000
    )
    return response["completion"]

additional_insights = generate_additional_insights(transcription_text)
print("Insights Adicionales:\n", additional_insights)
